# Search for orientation of image of HD160617 with astrometry.net


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNES, DESC-LSST fullmember, behalf AuxTel Teal VERA C. Rubin observatory
- creation date : October 5th 2021
- update : October 7th 2021 : run spectractor
- version v0

CCIN2P3:

- works with kernels **anaconda3_auxtel** (with libradtran) and **anaconda3_auxtel_v2** (no libradtran)
- works with kernel **python 3** locally 

# Scan

    # wide scan : HD 160617 
    # all_myseq_holo=range(234,310)
    # narrow scan :  HD 160617 
    # all_myseq_holo=range(317,365)


# Imports

In [41]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib import  transforms
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re

plt.rcParams["figure.figsize"] = (12,12)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [42]:
from iminuit import Minuit
import iminuit

In [43]:
iminuit.__version__

'2.7.0'

In [44]:
from astropy.io import fits
from spectractor import parameters
from spectractor.extractor.extractor import Spectractor
from spectractor.extractor.images import *
from scipy import ndimage, misc

In [45]:
from scipy.stats import kurtosis,skew
from scipy.stats import kurtosistest,skewtest,normaltest

In [46]:
from distutils.dir_util import copy_tree

In [47]:
from astropy.visualization import simple_norm

In [48]:
from astropy.coordinates import SkyCoord, EarthLocation, AltAz

In [49]:
from astroquery.simbad import Simbad
#import astropy.coordinates as coord

import astropy.units as u

In [50]:
from astropy.wcs import WCS
from astropy.io import fits
from astropy.table import Table

# Configuration

In [51]:
version="v4.0"

In [52]:
FLAG_REMOVE_WCS=False

In [53]:
FLAG_RUNSPECTRACTOR=False

# Path

In [54]:
# path index for each month
DATE="2021-07-07"
DATE2="2021_07_07"

In [55]:
# select if we run at CC or not (locally) 
HOSTCC=False

In [56]:
# Set path depending on which computer running (according HOSTCC)
if HOSTCC:
    path_auxtel="/sps/lsst/groups/auxtel"
    path_spectractor=os.path.join(path_auxtel,"softs/github/desc/Spectractor")
    path_spectractor_config=os.path.join(path_spectractor,"config")
    path_images=os.path.join(path_auxtel,"data/2021/holo/quickLookExp/"+DATE)
    path_images2=os.path.join(path_auxtel,"data/2021/holo/quickLookExp_v2/"+DATE)
    path_output_spectractor=os.path.join(path_auxtel,"data/2021/holo/OutputSpectractor/"+DATE)
    
else:
    path_auxtel="/Users/dagoret/DATA/AuxTelData2021"
    path_spectractor=os.path.join(path_auxtel,"/users/dagoret/MacOSX/github/LSST/Spectractor")
    path_spectractor_config=os.path.join(path_spectractor,"config")
    path_images=os.path.join(path_auxtel,"holo/quickLookExp/"+DATE)
    path_images2=os.path.join(path_auxtel,"holo/quickLookExp_v2/"+DATE)
    path_output_spectractor=os.path.join(path_auxtel,"holo/OutputSpectractor/"+DATE)

# Utility Functions

In [57]:
def file_tag_forsorting(filename):
    #m=re.findall('^Cor_holo4_003_.*([0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]_.*)[.]fits$',filename)
    m=re.findall('^holo4_003_*_(.*)_.*_quickLookExp[.]fits$',filename)
    #print(m)
    words=m[0].split("_")
    
    outputstring=words[1]
    
    if outputstring=="slew":
        outputstring+="_icrs"
    return outputstring

In [58]:
def file_target(filename):
    m=file_tag_forsorting(filename)
    if len(m)>0:
        return m
    elif re.search('NGC4755',filename):
        return ['NGC4755']
    else:
        return []
         

In [59]:
def is_nan(x):
    return (x != x)

# Logbook for input file selection

- the logbook contains all input image path and possibily the order 0 position

In [60]:
filename_logbook='logbooks/auxtelholologbook_'+DATE2+'_'+ version +'.csv'

In [61]:
df=pd.read_csv(filename_logbook,index_col=0)

In [62]:
pd.set_option('display.max_rows', None)

In [63]:
# show the list of input files from the logbook to select the file index in the logbook
df

,index,date,path,file,num,object,filter,disperser,airmass,exposure,...,temperature,humidity,windspd,Obj-posXpix,Obj-posYpix,Dx,Dy,skip,run,quality
0,0,2021-07-08T02:50:31.569,/Users/dagoret/DATA/AuxTelData2021/holo/quickL...,holo4_003_RG610_HD160617_20210707_000234_quick...,234,HD160617,RG610,holo4_003,1.022300,5.0,...,12.1,6.0,3.8,600.0,2000.0,NaN,NaN,,1.0,1.0
1,1,2021-07-08T02:51:12.101,/Users/dagoret/DATA/AuxTelData2021/holo/quickL...,holo4_003_RG610_HD160617_20210707_000235_quick...,235,HD160617,RG610,holo4_003,1.022020,5.0,...,12.1,6.0,4.4,600.0,2000.0,NaN,NaN,,1.0,1.0
2,2,2021-07-08T02:51:21.256,/Users/dagoret/DATA/AuxTelData2021/holo/quickL...,holo4_003_RG610_HD160617_20210707_000236_quick...,236,HD160617,RG610,holo4_003,1.021958,5.0,...,12.1,6.0,4.4,600.0,1900.0,NaN,NaN,,1.0,1.0
3,3,2021-07-08T02:51:30.934,/Users/dagoret/DATA/AuxTelData2021/holo/quickL...,holo4_003_RG610_HD160617_20210707_000237_quick...,237,HD160617,RG610,holo4_003,1.021893,5.0,...,12.1,6.0,4.4,600.0,1900.0,NaN,NaN,,1.0,1.0
4,4,2021-07-08T02:51:40.885,/Users/dagoret/DATA/AuxTelData2021/holo/quickL...,holo4_003_RG610_HD160617_20210707_000238_quick...,238,HD160617,RG610,holo4_003,1.021826,5.0,...,12.1,6.0,4.9,600.0,1900.0,NaN,NaN,,1.0,1.0
5,5,2021-07-08T02:51:50.381,/Users/dagoret/DATA/AuxTelData2021/holo/quickL...,holo4_003_RG610_HD160617_20210707_000239_quick...,239,HD160617,RG610,holo4_003,1.021763,5.0,...,12.1,6.0,4.9,600.0,1900.0,NaN,NaN,,1.0,1.0
6,6,2021-07-08T02:51:59.759,/Users/dagoret/DATA/AuxTelData2021/holo/quickL...,holo4_003_RG610_HD160617_20210707_000240_quick...,240,HD160617,RG610,holo4_003,1.021700,5.0,...,12.1,6.0,4.9,600.0,1900.0,NaN,NaN,,1.0,1.0
7,7,2021-07-08T02:52:10.113,/Users/dagoret/DATA/AuxTelData2021/holo/quickL...,holo4_003_RG610_HD160617_20210707_000241_quick...,241,HD160617,RG610,holo4_003,1.021631,5.0,...,12.1,6.0,4.9,600.0,1900.0,NaN,NaN,,1.0,1.0
8,8,2021-07-08T02:52:19.262,/Users/dagoret/DATA/AuxTelData2021/holo/quickL...,holo4_003_RG610_HD160617_20210707_000242_quick...,242,HD160617,RG610,holo4_003,1.021572,5.0,...,12.1,6.0,4.9,600.0,1900.0,NaN,NaN,,1.0,1.0
9,9,2021-07-08T02:52:29.022,/Users/dagoret/DATA/AuxTelData2021/holo/quickL...,holo4_003_RG610_HD160617_20210707_000243_quick...,243,HD160617,RG610,holo4_003,1.021508,5.0,...,12.1,6.0,4.9,600.0,1900.0,NaN,NaN,,1.0,1.0


# Selection of input file
- the input file is selected from the logbook list above

In [64]:
# wide scan : HD 160617 
# all_myseq_holo=range(234,310)

# narrow scan :  HD 160617 
# all_myseq_holo=range(317,365)

In [65]:
#idx=76
#idx=96
#idx=109
idx=76 # beginning of narrow scan

In [66]:
target_name=df.iloc[idx]['object']
disperser_name=df.iloc[idx]['disperser']
filename_name=df.iloc[idx]['file']

In [67]:
sky_coord_target=SkyCoord.from_name(target_name)
sky_coord_target

<SkyCoord (ICRS): (ra, dec) in deg
    (265.705515, -40.32097381)>

In [68]:
print(df.iloc[idx]['file'])

holo4_003_empty_HD160617_20210707_000317_quickLookExp.fits


In [69]:
print(df.iloc[idx])

index                                                         76
date                                     2021-07-08T04:49:32.013
path           /Users/dagoret/DATA/AuxTelData2021/holo/quickL...
file           holo4_003_empty_HD160617_20210707_000317_quick...
num                                                          317
object                                                  HD160617
filter                                                   empty_1
disperser                                              holo4_003
airmass                                                 1.065945
exposure                                                    20.0
focus                                                  -1.577194
seeing                                                       NaN
pressure                                                  742.25
temperature                                                 12.3
humidity                                                     6.0
windspd                  

In [70]:
myhome=os.getenv("HOME")

In [71]:
dir_images=path_images

In [72]:
filename_image=df['file'][idx]
filename_image

'holo4_003_empty_HD160617_20210707_000317_quickLookExp.fits'

In [73]:
file_target(filename_image)

'HD160617'

## If order 0 position exists in logbook it is selected, otherwise put it by hand

In [74]:
x0=df['Obj-posXpix'][idx]
y0=df['Obj-posYpix'][idx]

In [75]:
FLAG_ORDER0_LOCATION=False

In [76]:
if not is_nan(x0)and not is_nan(y0):
    FLAG_ORDER0_LOCATION=True
    print("Order 0 location from logbook : ({},{})".format(x0,y0))
else:
    print("NO Order 0 location from logbook ! ")      

Order 0 location from logbook : (500.0,1900.0)


## Spectractor Configuration 

- Usually the Spectractor configuration file is put in local dir **./config**


Ma suggestion, pour l'étude du centre optique via les scans des hologrammes, est de **mettre SPECTRACTOR_COMPUTE_ROTATION_ANGLE sur False** . 

Comme les angles ne sont pas très grands, si les marges du rectangle sont assez larges, réglables avec : 

    [background subtraction parameters] 
    # half transverse width of the signal rectangular window in pixels 
    PIXWIDTH_SIGNAL = 40 

alors le spectrogramme devrait tenir dans une boite rectangulaire même sans rotation de l'image. **L'important est de garder SPECTRACTOR_DECONVOLUTION_FFM à True car c'est lui qui te donnera l'angle de l'axe de dispersion ajusté sur les données, dont la valeur sera dans le mot clé ROTANGLE du header de sortie**. 


In [77]:
filename_image.split(".")[0]

'holo4_003_empty_HD160617_20210707_000317_quickLookExp'

In [78]:
path_output_spectractor

'/Users/dagoret/DATA/AuxTelData2021/holo/OutputSpectractor/2021-07-07'

In [79]:

# Fullfilename
filename=os.path.join(dir_images,filename_image)


# subdirectory
subdir=filename_image.split(".")[0]

# final output directory (where results will be copied to be backed up at the end)
finalpath_output_spectractor=os.path.join(path_output_spectractor,subdir)

# local directories to put spectra and plots
output_directory="./outputs_process_holo_scan"
output_figures="figures_process_holo_scan"

# Final output directory


guess = [250,2600]
disperser_label = "holo4_003"
# old version
# config = os.path.join(path_spectractor_config,"auxtel_quicklook.ini")
# new version (September 20th 2021)

# configuration

# standard spectractor init configuration
#config = os.path.join(path_spectractor_config,"auxtel.ini")
# special for scan in XY
config="./config/auxtel_scanXY.ini"

target = file_target(filename_image)[0]

In [80]:
finalpath_output_spectractor

'/Users/dagoret/DATA/AuxTelData2021/holo/OutputSpectractor/2021-07-07/holo4_003_empty_HD160617_20210707_000317_quickLookExp'

# Configuration of the Spectractor running mode

In [81]:
parameters.debug=True
parameters.verbose=True
parameters.display=True
parameters.LIVE_FIT=False

In [82]:
#parameters.DISPERSER_DIR = os.path.join(spectractor_dir, "extractor/dispersers/")
#parameters.CONFIG_DIR = os.path.join(spectractor_dir, "../config/")
#parameters.THROUGHPUT_DIR = os.path.join(spectractor_dir, "simulation/CTIOThroughput/")

# Transform the input file

- the image fits format extracted from quickLookExp images at NCSA has not the exact format required by Spectractor.

In [83]:
from shutil import copyfile

In [84]:
copyfile(filename,filename_image)

'holo4_003_empty_HD160617_20210707_000317_quickLookExp.fits'

In [85]:
from astropy.io import fits

In [86]:
hdu = fits.open(filename_image)

In [87]:
hdu.info()

Filename: holo4_003_empty_HD160617_20210707_000317_quickLookExp.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  Segment10     1 PrimaryHDU     124   (4072, 4000)   float32   


In [88]:
# extract header and the image
header=hdu[0].header
image=hdu[0].data

In [89]:
hdu.close()

In [95]:
plt.imshow(image, origin ="lower",norm =  LogNorm(vmin=0.001,vmax=.0))

ValueError: minvalue must be less than or equal to maxvalue

<Figure size 864x864 with 1 Axes>

In [90]:
image.shape

(4000, 4072)

In [91]:
assert False

AssertionError: 

## Rotate image

- rotate the image according what is expected in Spectractor

In [ ]:
#rot_image=np.flip(np.flip(image, 1), 0)
rot_image=np.flip(image, 0)

### Remove WCS

- may not be necessary

In [ ]:
if FLAG_REMOVE_WCS:
    del header['WCSAXES']
    del header['CTYPE1']
    del header['CTYPE2']
    del header['CUNIT1']
    del header['CUNIT2']
    del header['CRVAL1']
    del header['CRVAL2']
    del header['CRPIX1']
    del header['CRPIX2']

### Put HASTART in hours units

In [ ]:
header["HASTART"]=header["HAEND"]  # Spectractor search for HASTART in Hour

In [ ]:
fig=plt.figure(figsize=(8,8))
ax=fig.add_subplot(1,1,1)
ax.imshow(rot_image,origin="lower",norm=LogNorm(0.1,1000))
ax.grid()
ax.set_title("image pre-rotated for spectractor")
plt.show()

## Reshape the file for load_image_AUXTEL() in Spectractor/spectractor/extractor/images.py

In [ ]:
primary_hdu = fits.PrimaryHDU(header=header)
image_hdu = fits.ImageHDU(rot_image)

In [ ]:
hdu_list = fits.HDUList([primary_hdu, image_hdu])

In [ ]:
hdu_list.info()

In [ ]:
hdu_list.writeto(filename_image,overwrite=True)

# Check new version

In [ ]:
filename_image

In [ ]:
fullfilename2=os.path.join( path_images2,filename_image)

In [ ]:
hdu2 = fits.open(fullfilename2)

In [ ]:
hdu2.info()

In [ ]:
# extract header and the image
header2=hdu2[0].header
image2=hdu2[1].data

In [ ]:
fig=plt.figure(figsize=(8,8))
ax=fig.add_subplot(1,1,1)
ax.imshow(image2,origin="lower",norm=LogNorm(0.1,1000))
ax.grid()
ax.set_title("image pre-rotated (at NCSA) for spectractor")
plt.show()

## Load the image in the new image file with Spectractor

In [ ]:
#image=Image(file_name=filename, target_label=target, disperser_label=disperser_label, config=config)
image=Image(file_name=fullfilename2, target_label=target, disperser_label=disperser_label, config=config)

## Show relevant parameters

In [ ]:
parameters.OBS_NAME

In [ ]:
parameters.DISPERSER_DIR

In [ ]:
parameters.DISTANCE2CCD

In [ ]:
parameters.LAMBDA_MIN 

In [ ]:
parameters.LAMBDA_MAX 

In [ ]:
parameters.LSST_SAVEFIGPATH=True
parameters.LSST_SAVEFIGPATH=output_figures

In [ ]:
image.filter_label

In [ ]:
image.header["FILTER"]

In [ ]:
image.header

# Plot image and find the target position

- this image plotting is used to find the order 0 location to be set in logbook

- for the moment this is humanly done

## Load the new image file with Spectractor

In [ ]:
title="{}) {}".format(idx,filename_image)
image.plot_image(figsize=(12, 10),scale="log",title=title)

In [ ]:
# with filter
guess = [600., 2100.] # filename_image=df['file'][0] , holo4_003_RG610_HD160617_20210707_000234_quickLookExp.fits
guess = [600., 2100.] # filename_image=df['file'][1] , holo4_003_RG610_HD160617_20210707_000235_quickLookExp.fits
guess = [600., 2100.] # filename_image=df['file'][2] , holo4_003_RG610_HD160617_20210707_000236_quickLookExp.fits
guess = [600., 2100.] # filename_image=df['file'][3] , holo4_003_RG610_HD160617_20210707_000237_quickLookExp.fits
guess = [600., 2100.] # filename_image=df['file'][4] , holo4_003_RG610_HD160617_20210707_000238_quickLookExp.fits
guess = [600., 2100.] # filename_image=df['file'][5] , holo4_003_RG610_HD160617_20210707_000239_quickLookExp.fits
guess = [600., 2100.] # filename_image=df['file'][6] , holo4_003_RG610_HD160617_20210707_000240_quickLookExp.fits
guess = [600., 2100.] # filename_image=df['file'][7] , holo4_003_RG610_HD160617_20210707_000241_quickLookExp.fits
guess = [600., 2100.] # filename_image=df['file'][8] , holo4_003_RG610_HD160617_20210707_000242_quickLookExp.fits
guess = [600., 2100.] # filename_image=df['file'][9] , holo4_003_RG610_HD160617_20210707_000243_quickLookExp.fits
guess = [1300., 700.] # filename_image=df['file'][10] , holo4_003_RG610_HD160617_20210707_000244_quickLookExp.fits
guess = [250., 1800.] # filename_image=df['file'][11] , holo4_003_RG610_HD160617_20210707_000245_quickLookExp.fits
guess = [250., 1800.] # filename_image=df['file'][12] , holo4_003_RG610_HD160617_20210707_000246_quickLookExp.fits
guess = [250., 1800.] # filename_image=df['file'][13] , holo4_003_RG610_HD160617_20210707_000247_quickLookExp.fits


# wide scan
guess = [1400., 700.] # filename_image=df['file'][14] , holo4_003_empty_HD160617_20210707_000248_quickLookExp.fits
guess = [1400., 800.] # filename_image=df['file'][15] , holo4_003_empty_HD160617_20210707_000249_quickLookExp.fits
guess = [400., 1800.] # filename_image=df['file'][16] , holo4_003_empty_HD160617_20210707_000250_quickLookExp.fits
guess = [400., 1800.] # filename_image=df['file'][17] , holo4_003_empty_HD160617_20210707_000251_quickLookExp.fits
guess = [400., 1800.] # filename_image=df['file'][18] , holo4_003_empty_HD160617_20210707_000252_quickLookExp.fits
#guess = [50., 800.] # filename_image=df['file'][19] , holo4_003_empty_HD160617_20210707_000253_quickLookExp.fits
#guess = [50., 800.] # filename_image=df['file'][20] , holo4_003_empty_HD160617_20210707_000254_quickLookExp.fits
#guess = [50., 800.] # filename_image=df['file'][21] , holo4_003_empty_HD160617_20210707_000255_quickLookExp.fits

#

# narrrow scan

guess = [500.,2100.] # filename_image=df['file'][76] , 'holo4_003_empty_HD160617_20210707_000317_quickLookExp.fits'
guess = [500.,2100.] # filename_image=df['file'][77] , 'holo4_003_empty_HD160617_20210707_000318_quickLookExp.fits'
guess = [500.,1800.] # filename_image=df['file'][78] , 'holo4_003_empty_HD160617_20210707_000319_quickLookExp.fits'
guess = [500.,1800.] # filename_image=df['file'][79] , 'holo4_003_empty_HD160617_20210707_000320_quickLookExp.fits'
guess = [500.,1300.] # filename_image=df['file'][80] , 'holo4_003_empty_HD160617_20210707_000321_quickLookExp.fits'

- note now the position are written in the make_logbook.ipynb notebook

## Set the 0th order location to be provided to Spectractor

In [ ]:
filename_image

In [ ]:
# overwrite if localtion is taken from that in logbook
if FLAG_ORDER0_LOCATION : 
    print("Set Order 0 location from logbook : ({},{})".format(x0,y0))
    guess = [x0,y0]

# Open image in astropy

In [ ]:
sky_coord_target

In [ ]:
hdu = fits.open(filename_image)

In [ ]:
header=hdu[0].header
image=hdu[1].data

In [ ]:
wcs = WCS(header)

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(projection=wcs)
plt.imshow(image, aspect='equal',norm=LogNorm(0.1,1000))
plt.xlabel(r'RA')
plt.ylabel(r'Dec')

overlay = ax.get_coords_overlay('icrs')
overlay.grid(color='white', ls='dotted')

# Work with astrometry.net

In [ ]:
FLAG_RESULTS_ASTROMETRY_NET=True
#DIR_RESULT_ASTROMETRY_NET="astrometry_HD160617"
DIR_RESULT_ASTROMETRY_NET="astrometry_HD160617_20210707_000317"

In [ ]:
file_new_image=fullfilename2 # New FITS image
file_axy="axy.fits"             # Stars detected in your images (x,y table) 
file_wcs="wcs.fits"             # WCS file
file_corr="corr.fits"           # Correspondences between image and reference stars
file_rdls="rdls.fits"           # Reference stars nearby (RA,Dec table):

In [ ]:
fullfile_new_image=os.path.join(DIR_RESULT_ASTROMETRY_NET,file_new_image)
fullfile_wcs=os.path.join(DIR_RESULT_ASTROMETRY_NET,file_wcs)
fullfile_axy=os.path.join(DIR_RESULT_ASTROMETRY_NET,file_axy)
fullfile_rdls=os.path.join(DIR_RESULT_ASTROMETRY_NET,file_rdls)
fullfile_corr=os.path.join(DIR_RESULT_ASTROMETRY_NET,file_corr)

## The astrometry.net image

In [ ]:
if FLAG_RESULTS_ASTROMETRY_NET:
    hdu = fits.open(fullfile_new_image)
    hdu.info()

In [ ]:
if FLAG_RESULTS_ASTROMETRY_NET:
    header=hdu[0].header
    image=hdu[0].data
    wcs = WCS(header)
    print(wcs)

In [ ]:
header["HA"]

In [ ]:
if FLAG_RESULTS_ASTROMETRY_NET:
    fig = plt.figure(figsize=(10, 10))
    ax = plt.subplot(111)
    ax.imshow(image, origin="lower",aspect='equal',norm=LogNorm(0.1,1000))
    ax.set_xlabel(r' X pixel')
    ax.set_ylabel(r' Y pixel')


    ax.grid(color='white', ls='dotted')
    ax.set_title("image in pixel frame")

In [ ]:
if FLAG_RESULTS_ASTROMETRY_NET:
    fig = plt.figure(figsize=(10, 10))
    ax = plt.subplot(projection=wcs)
    ax.imshow(image, aspect='equal',norm=LogNorm(0.1,1000))
    ax.set_xlabel(r'RA')
    ax.set_ylabel(r'Dec')

    overlay = ax.get_coords_overlay('icrs')
    overlay.grid(color='white', ls='dotted')
    
    plt.suptitle("image in celestial coordinates",fontsize=20)
    

## WCS coordinates

In [ ]:
if FLAG_RESULTS_ASTROMETRY_NET:
    hdu_wcs = fits.open(fullfile_wcs)
    hdu_wcs.info()
    hdu = hdu_wcs[0]
    wcs_astrometry = WCS(hdu.header)
    print(wcs_astrometry)

## Stars detected in your images (x,y table) 

In [ ]:
if FLAG_RESULTS_ASTROMETRY_NET:
    hdu_axy = fits.open(fullfile_axy)
    hdu_axy.info()

In [ ]:
if FLAG_RESULTS_ASTROMETRY_NET:
    t_axy = Table.read(fullfile_axy, hdu=1)  

In [ ]:
if FLAG_RESULTS_ASTROMETRY_NET:
    t_axy.show_in_notebook()
    print(t_axy)

##  Reference stars nearby (RA,Dec table):

In [ ]:
if FLAG_RESULTS_ASTROMETRY_NET:
    hdu_rdls = fits.open(fullfile_rdls)
    hdu_rdls.info()

In [ ]:
if FLAG_RESULTS_ASTROMETRY_NET:
    t_rdls = Table.read(fullfile_rdls, hdu=1)  

In [ ]:
if FLAG_RESULTS_ASTROMETRY_NET:
    t_rdls.show_in_notebook()
    print(t_rdls)

In [ ]:
NrefStars=len(t_rdls)

In [ ]:
all_sep=np.zeros(NrefStars)
for idx in np.arange(NrefStars):
    the_ra=t_rdls[idx]['RA']
    the_dec=t_rdls[idx]['DEC']
    skyc=SkyCoord(ra=the_ra, dec=the_dec,frame='icrs',unit=(u.deg, u.deg))
    sep = sky_coord_target.separation(skyc).arcsec 
   
    all_sep[idx]=sep
    
    print(">>>>>>>>>>",the_ra,the_dec,sep)
    result_table = Simbad.query_region(skyc,radius=5./3600. * u.deg)
    if result_table != None :
        print(result_table)
    else:
        print("Reference star not found")
       

In [ ]:
all_sep

In [ ]:
idx_target=np.where(all_sep==np.min(all_sep))[0][0]
idx_target

## Correspondences between image and reference stars

In [ ]:
if FLAG_RESULTS_ASTROMETRY_NET:
    hdu_corr = fits.open(fullfile_corr)
    hdu_corr.info()

In [ ]:
if FLAG_RESULTS_ASTROMETRY_NET:
    t_corr = Table.read(fullfile_corr, hdu=1)  

In [ ]:
if FLAG_RESULTS_ASTROMETRY_NET:
    t_corr.show_in_notebook()
    print(t_corr)

In [ ]:
t_corr.show_in_notebook()

In [ ]:
if FLAG_RESULTS_ASTROMETRY_NET:
    fig = plt.figure(figsize=(10, 10))
    ax = plt.subplot(projection=wcs)
    ax.imshow(image, aspect='equal',norm=LogNorm(0.1,1000))
    ax.set_xlabel(r'RA')
    ax.set_ylabel(r'Dec')

    overlay = ax.get_coords_overlay('icrs')
    overlay.grid(color='white', ls='dotted')

    ax.scatter(sky_coord_target.ra.deg,sky_coord_target.dec.deg, transform=ax.get_transform('icrs'), s=300,edgecolor='red', facecolor='none',label="target catalog location")
    ax.scatter(t_rdls[idx_target]["RA"],t_rdls[idx_target]["DEC"], transform=ax.get_transform('icrs'), s=300,edgecolor='blue', facecolor='none',label="closest ref star location in astrometry")

    ax.legend()
    
    plt.suptitle("image in celestial coordinates",fontsize=20)

In [ ]:
NrefStars=len(t_corr)
NrefStars

In [ ]:
all_sep=np.zeros(NrefStars)+9999
for idx in np.arange(NrefStars):
    
    the_ra=t_corr[idx]['field_ra']
    the_dec=t_corr[idx]['field_dec']
    skyc=SkyCoord(ra=the_ra, dec=the_dec,frame='icrs',unit=(u.deg, u.deg))
    sep = sky_coord_target.separation(skyc).arcsec 
   
    all_sep[idx]=sep
    
    print(">>>>>>>>>>",idx," ra=",the_ra," dec=",the_dec," sep=",sep)
    result_table = Simbad.query_region(skyc,radius=5./3600. * u.deg)
    if result_table != None :
        print(result_table)
    else:
        print("Reference star not found")
       

In [ ]:
all_sep

In [ ]:
if FLAG_RESULTS_ASTROMETRY_NET:
    fig = plt.figure(figsize=(10, 10))
    ax = plt.subplot(111)
    ax.imshow(image, origin="lower",aspect='equal',norm=LogNorm(0.1,1000))
    ax.set_xlabel(r' X pixel')
    ax.set_ylabel(r' Y pixel')

    x0, y0 = wcs.world_to_pixel(sky_coord_target)
    ax.scatter(x0,y0, s=300,edgecolor='red', facecolor='none',label="target location after astrometry")

    ax.grid(color='white', ls='dotted')
    ax.set_title("image in pixel frame")
    ax.legend()

# Process Spectractor

In [ ]:
parameters.debug=True
parameters.verbose=True
parameters.display=True
parameters.LIVE_FIT=False

In [ ]:
# Fullfilename
filename=os.path.join(dir_images,filename_image)


# subdirectory
subdir=filename_name.split(".")[0]


# local directories to put spectra and plots
output_directory="./outputs_process_holo_scan_orient"
output_figures="./figures_process_holo_scan_orient"

# Final output directory


guess = [500,2000]
disperser_label = "holo4_003"
# old version
# config = os.path.join(path_spectractor_config,"auxtel_quicklook.ini")
# new version (September 20th 2021)

# configuration

# standard spectractor init configuration
#config = os.path.join(path_spectractor_config,"auxtel.ini")
# special for scan in XY
config="./config/auxtel_scanXY.ini"

target = file_target(filename_image)[0]

In [ ]:
! ls -l $fullfile_new_image

In [ ]:
from shutil import copyfile
copyfile(fullfile_new_image,filename_name)
from astropy.io import fits
hdu = fits.open(filename_name)
hdu.info()
# extract header and the image
header=hdu[0].header
image=hdu[0].data

hdu.close()


primary_hdu = fits.PrimaryHDU(header=header)
image_hdu = fits.ImageHDU(rot_image)
hdu_list = fits.HDUList([primary_hdu, image_hdu])
hdu_list.info()
hdu_list.writeto(filename_name,overwrite=True)

In [ ]:
header

In [ ]:
if FLAG_RUNSPECTRACTOR:
    #image=Image(file_name=filename, target_label=target, disperser_label=disperser_label, config=config)
    image=Image(file_name=filename_name, target_label=target_name, disperser_label=disperser_name, config=config)

In [ ]:
if FLAG_RUNSPECTRACTOR:
    title="{}) {}".format(idx,filename_name)
    image.plot_image(figsize=(12, 10),scale="log",title=title)

In [ ]:
if FLAG_RUNSPECTRACTOR:
    parameters.VERBOSE = True
    parameters.DEBUG = True
    #x1, y1 = find_target(image, guess,rotated=False, use_wcs=False)
    x1, y1 = find_target(image, guess,rotated=False)

In [ ]:
if FLAG_RUNSPECTRACTOR:
    spectrum = Spectractor(filename_name, output_directory, guess=[x1,y1], target_label=target_name, disperser_label=disperser_name, config=config)